# Tinyflux: Example with fictious bank transactions

In [1]:
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

## Initialize the TinyFlux database

In [2]:
db = TinyFlux("swiss_bank_transactions.db")

## Writing a CSV file into Tinyflux

In [3]:
# Read CSV and insert data into TinyFlux
with open("example_data/swiss_bank_transactions.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
            measurement="transactions",
            fields={"amount": float(row["amount"])},  # Only numeric fields
            tags={
                "location": row["location"],
                "account_id": row["account_id"],
                "currency": row["currency"],  # Move non-numeric fields to tags
                "transaction_type": row["transaction_type"]
            }
        )
        db.insert(point)

Bad pipe message: %s [b'0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nHost: localhost:40933\r\nUs', b'-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.']
Bad pipe message: %s [b'0.0 Safari/537.36\r\nAccept-Encoding: gzip, defla']
Bad pipe message: %s [b', br, zstd\r\nAccept-Language: de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7\r\nCache-Control: max-age=0\r\nReferer: https://fuzzy-', b'mputing-machine-6949wrqg476p3gvq.github.dev/\r\nX-Request-ID: 4d1156bb3e98dce35124b1dc78d2762f\r\nX-Re', b'-IP: 10.240.0.69\r\nX-Forwarded-Port: 443\r\nX-Forwarded-Scheme: https\r\nX-Original-URI: /\r\nX-Scheme:']
Bad pipe message: %s [b'ttps\r\nsec-fetch-site: same-site']
Bad pipe message: %s [b'sec-fetch-mo']
Bad pipe message: %s [b': navigate\r\nsec-fetch-dest: document\r\nsec-ch-ua: "Chromium";v="128", "Not;A=Brand";v="24", "Google ', b'rome";v="128"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"']
Bad pipe

## Writing manually into Tinyflux

In [4]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 1000.0}
)

p2 = Point(
    time=datetime(2024, 1, 2, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 2000.0}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

2

## Querying by Time

In [14]:
# Search for a time value
Time = TimeQuery()
time_start = Time >= datetime(2023, 1, 1, tzinfo=timezone.utc)
time_end = Time < datetime(2023, 12, 31, tzinfo=timezone.utc)
time_end_2 = db.get(time_end)
time_count = db.count(time_start & time_end)
print("\nTime Query Count:")
print(time_count)



Time Query Count:
1000


## Querying by Field

In [35]:
# Search for a field value
Field = FieldQuery()
#Tag = TagQuery()
#field_results = db.select("fields.amount", Field.amount > 1000.0)
field_results = db.search(Field.amount < 1000.0)
#field_select = field_results.select("tag.location","tag.account")
#selected_results = field_results.select("tag.location", "tag.account", "fields.amount")

print("\nField Query Results:")
for result in field_results:
    # Assuming 'result' is an object with attributes like 'tag' and 'fields'
    
    # Safely access attributes using `getattr()` to handle missing values
    location = getattr(result.tag, "location", "N/A")
    account = getattr(result.tag, "account", "N/A")
    amount = getattr(result.fields, "amount", "N/A")
    
    # Display the specific fields
    print(f"Location: {location}, Account: {account}, Amount: {amount}")




#print("\nField Query Results:")
#for result in selected_results:
#    print(result)


Field Query Results:


AttributeError: 'Point' object has no attribute 'tag'

## Querying by Tag

In [16]:
# Search for a tag value
Tag = TagQuery()
tag_results = db.search(Tag.location == 'Zurich')
print("Tag Query Results:")
for result in tag_results:
    print(result)

Tag Query Results:
Point(time=2023-01-01T12:29:00+00:00, measurement=transactions, tags=location:Zurich; account_id:6265; currency:USD; transaction_type:deposit, fields=amount:1084.57)
Point(time=2023-01-01T15:46:00+00:00, measurement=transactions, tags=location:Zurich; account_id:2689; currency:EUR; transaction_type:deposit, fields=amount:219.92)
Point(time=2023-01-01T18:10:00+00:00, measurement=transactions, tags=location:Zurich; account_id:1431; currency:USD; transaction_type:transfer, fields=amount:2738.98)
Point(time=2023-01-02T00:51:00+00:00, measurement=transactions, tags=location:Zurich; account_id:7491; currency:USD; transaction_type:transfer, fields=amount:1794.01)
Point(time=2023-01-02T09:14:00+00:00, measurement=transactions, tags=location:Zurich; account_id:4250; currency:EUR; transaction_type:withdrawal, fields=amount:150.51)
Point(time=2023-01-02T14:54:00+00:00, measurement=transactions, tags=location:Zurich; account_id:1067; currency:CHF; transaction_type:transfer, fiel